# Skin Cancer Classification Challenge

**Summary : ** skin cancer.

## Preliminaries and Imports

In [ ]:
#general & system\n",
import os

#data processing
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
plt.rcParams['figure.figsize'] = 10, 10  #default setting
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

#data augmentation
from PIL import Image
from random import choice
import cv2
import numpy as np
#import keras.preprocessing.image as prep

#ML part
from keras.applications import inception_v3
from keras.applications.inception_v3 import preprocess_input
from keras.preprocessing import image as kimage
from keras.models import Sequential
from keras.layers import GlobalMaxPooling2D, Dense, BatchNormalization, GlobalAveragePooling2D, Dropout,Activation,Flatten
from keras.models import Model
from keras.optimizers import SGD
from keras.models import load_model
from keras.preprocessing.image import ImageDataGenerator

#validation
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight

%matplotlib inline

## I. Load Data

- **Rq : ** Datasets are quite light, can be fully loaded in a laptop memory with ease.

In [ ]:
%%time
meta = pd.read_csv("data/metadata.csv")

In [ ]:
%%time
filelist = os.listdir("data/resized/")
X_train = np.array([np.array(Image.open("data/resized/"+fname)) for fname in filelist])

In [ ]:
X_train.shape

In [ ]:
#get target
filelist = [x.replace(".jpg","") for x in filelist]
strates = [meta[meta["name"]==x]["meta_clinical_benign_malignant"].values[0] for x in filelist]
Y_train = [1 if x=="malignant" else 0 for x in strates]

In [ ]:
#control
print(filelist[0:5])
print(Y_train[0:5])
print(strates[0:5])

## II. Data Preparation

- **Resampling**

- **Class weights**

In [ ]:
class_weight = class_weight.compute_class_weight('balanced', np.unique(Y_train), Y_train)

- **Data splitting**

In [ ]:
#data splitting
X_train, X_valid, Y_train, Y_valid = train_test_split(X_train, Y_train, random_state=42, train_size=0.80,
                                                      stratify = strates)

## III. Deep Learning with Transfer Learning

In [ ]:
weights_path = 'model/checkpoints/'

if not os.path.exists(weights_path):
    os.makedirs(weights_path)
    
epochs = 500
batch_size = 10

- **Using InceptionV3 : **

In [ ]:
#load base model
base_model = inception_v3.InceptionV3(weights='imagenet', include_top=False, input_shape=(299,299,3))
base_model.summary()

In [ ]:
model = Model(input=base_model.input, output=base_model.get_layer('flatten').output)

## IV. Evaluate Model

In [ ]:
#compute log loss
pred = model.predict(X_valid)
log_loss(Y_valid,pred)